In [ ]:
!pip install google-cloud-bigquery
from google.cloud import bigquery

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/credentials-file.json"

In [61]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

# Autenticar o usuário
auth.authenticate_user()

# Configurações do projeto e localização
project_id = "tcmodulo03"
location = "us-central1"

# Criando o cliente do BigQuery após autenticação
client = bigquery.Client(project=project_id, location=location)

# Habilitar formatação de tabelas
data_table.enable_dataframe_formatter()

# Agora você pode executar as consultas


AuthorizationError: Failed to fetch user credentials

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Configura o cliente do BigQuery
project_id = "tcmodulo03"
location = "us-central1"
client = bigquery.Client(project=project_id, location=location)

# Query ajustada
query = """
SELECT
  view_dados.A001,
  associativa.Descricao AS tipo_respondente
FROM
  `tcmodulo03.PNAD_COVID.view_dados_PNAD` AS view_dados
LEFT JOIN
  `tcmodulo03.PNAD_COVID.condicaoDomicilio` AS associativa
ON
  view_dados.A001A = CAST(associativa.Cod AS INT64)
"""

# Executa a query e salva no DataFrame
df = client.query(query).to_dataframe()

# Visualiza as primeiras linhas do DataFrame
print(df.head())


In [ ]:
# Contagem dos tipos de respondentes
respondente_counts = df["tipo_respondente"].value_counts().reset_index()
respondente_counts.columns = ["tipo_respondente", "quantidade"]

# Visualiza a contagem
print(respondente_counts)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib.ticker import FuncFormatter

# Calcular o percentual de cada tipo de respondente
respondente_counts["percentual"] = (respondente_counts["quantidade"] /
                                    respondente_counts["quantidade"].sum()) * 100

# Selecionar as Top 10 categorias
top_respondentes = respondente_counts.nlargest(10, "quantidade")

# Criar a categoria "Outros" para os menores volumes
outros = pd.DataFrame({
    "tipo_respondente": ["Outros"],
    "quantidade": respondente_counts["quantidade"].sum() - top_respondentes["quantidade"].sum(),
    "percentual": 100 - top_respondentes["percentual"].sum()
})

# Combinar Top 10 com a categoria "Outros"
grafico_data = pd.concat([top_respondentes, outros], ignore_index=True)

# Configuração do gráfico
fig, ax = plt.subplots(figsize=(10, 6))

# Paleta de cores personalizada
colors = sns.color_palette("viridis", len(grafico_data))

# Criar gráfico de barras
bars = ax.bar(grafico_data["tipo_respondente"],
              grafico_data["quantidade"], color=colors)

# Formatar o eixo Y para milhares (K)
def formatar_eixo_y(valor, pos):
    if valor >= 1_000_000:  # Milhões
        return f'{valor/1_000_000:.1f}M'
    elif valor >= 1_000:    # Milhares
        return f'{valor/1_000:.0f}K'
    return f'{int(valor)}'

ax.yaxis.set_major_formatter(FuncFormatter(formatar_eixo_y))

# Remover rótulos do eixo X
ax.set_xticklabels([])

# Título e rótulos
ax.set_title("Top 10 Tipos de Respondentes por Quantidade (com Outros)")
ax.set_ylabel("Quantidade")

# Criar a legenda ao lado com percentuais
handles = [Patch(facecolor=colors[i],
                 label=f"{grafico_data['tipo_respondente'].iloc[i]}: {grafico_data['percentual'].iloc[i]:.1f}%")
           for i in range(len(grafico_data))]

# Posicionar a legenda fora do gráfico, à direita
ax.legend(handles=handles, title="Tipo de Respondente",
          bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.)

# Ajustar layout
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Recuperar os dados da view no BigQuery
query = """
SELECT
  tipo_sintoma,
  `total_por_tipo`,
  percentual
FROM `tcmodulo03.PNAD_COVID.view_analise_sintomas`
"""
df_sintomas = client.query(query).to_dataframe()

# Ordenar os sintomas pelo percentual
df_sintomas = df_sintomas.sort_values(by="percentual", ascending=False)

# Configurar o tamanho da figura
fig, ax = plt.subplots(figsize=(10, 8))

# Paleta de cores
colors = sns.color_palette("viridis", len(df_sintomas))

# Gráfico de barras horizontais
bars = ax.barh(df_sintomas["tipo_sintoma"], df_sintomas["percentual"], color=colors)

# Inverter a ordem do eixo Y (para o mais alto no topo)
ax.invert_yaxis()

# Adicionar os percentuais ao lado das barras
for bar, percentual in zip(bars, df_sintomas["percentual"]):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
            f"{percentual:.1f}%", ha="left", va="center")

# Título e rótulos
ax.set_title("Distribuição dos Sintomas Reportados")
ax.set_xlabel("Percentual (%)")
ax.set_ylabel("Tipo de Sintoma")

# Remover linhas desnecessárias
sns.despine(left=True, bottom=True)

# Ajustar layout
plt.tight_layout()
plt.show()


In [ ]:
# Importar bibliotecas necessárias
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Autenticar e inicializar cliente BigQuery
auth.authenticate_user()
project_id = "tcmodulo03"
client = bigquery.Client(project=project_id)

# Query para obter os dados
query_teste = """
SELECT
  realizou_teste,
  total_respostas,
  percentual
FROM
  `tcmodulo03.PNAD_COVID.view_realizouTeste`
"""

# Executar a query e converter para DataFrame
df_teste = client.query(query_teste).to_dataframe()

# Arredondar o percentual para duas casas decimais
df_teste["percentual"] = df_teste["percentual"].round(2)

# Exibir os resultados em formato de tabela no Colab
print("Distribuição de Respostas sobre Realização de Teste:")
display(df_teste)


In [ ]:
# Importar bibliotecas necessárias
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick  # Importar formatador do eixo

# Autenticar e inicializar cliente BigQuery
auth.authenticate_user()
project_id = "tcmodulo03"
client = bigquery.Client(project=project_id)

# Query para buscar os resultados dos testes
query_resultado_teste = """
SELECT
  descricao AS resultado_teste,
  total_respostas,
  percentual
FROM
  `tcmodulo03.PNAD_COVID.view_resultadoTeste`
ORDER BY
  total_respostas DESC
"""

# Executar a query e armazenar no DataFrame
df_resultado_teste = client.query(query_resultado_teste).to_dataframe()

# Arredondar percentual para melhor apresentação
df_resultado_teste["percentual"] = df_resultado_teste["percentual"].round(2)

# Configuração do gráfico
sns.set(style="whitegrid")
fig, ax = plt.subplots(figsize=(10, 6))

# Criar gráfico de barras horizontais com paleta personalizada do Seaborn
colors = sns.color_palette("Set2", len(df_resultado_teste))
bars = ax.barh(df_resultado_teste["resultado_teste"],
               df_resultado_teste["total_respostas"],
               color=colors)

# Adicionar os valores no final das barras
for bar, percentual in zip(bars, df_resultado_teste["percentual"]):
    plt.text(bar.get_width() + 500, bar.get_y() + bar.get_height()/2,
             f"{percentual}%", va='center', fontsize=10)

# Formatador do eixo X (milhares = K, milhões = M)
def formatar_eixo_x(valor, pos):
    if valor >= 1_000_000:  # Milhões
        return f'{valor/1_000_000:.1f}M'
    elif valor >= 1_000:    # Milhares
        return f'{valor/1_000:.0f}K'
    return f'{int(valor)}'

ax.xaxis.set_major_formatter(mtick.FuncFormatter(formatar_eixo_x))

# Títulos e rótulos
ax.set_title("Distribuição dos Resultados dos Testes de COVID", fontsize=14)
ax.set_xlabel("Total de Respostas", fontsize=12)
ax.set_ylabel("Resultado do Teste", fontsize=12)

# Ajustar layout
plt.tight_layout()
plt.show()


In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar o nome do projeto explicitamente
PROJECT_ID = 'tcmodulo03'

# Criar o cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Query para obter dados dos casos positivos por estado
query = """
SELECT
  estado,
  total_casos,
  percentual
FROM `tcmodulo03.PNAD_COVID.view_casos_positivos_por_estado`
"""

try:
    # Executar a query e converter para DataFrame
    df_estados = client.query(query).to_dataframe()

    # Exibir a tabela detalhada
    print("Tabela detalhada de casos positivos por estado:")
    display(df_estados)

    # Configuração do estilo do gráfico
    sns.set_theme(style="whitegrid", palette="coolwarm")  # Paleta ajustada para "coolwarm"

    # Ordenar os dados para visualização
    df_estados = df_estados.sort_values(by="total_casos", ascending=False)

    # Criar o gráfico de barras
    plt.figure(figsize=(12, 6))
    sns.barplot(x="total_casos", y="estado", data=df_estados, orient="h", palette="coolwarm")

    # Configurar os rótulos do gráfico
    plt.xlabel("Total de Casos")
    plt.ylabel("Estado")
    plt.title("Casos Positivos por Estado - Regiões Mais Impactadas")

    # Exibir os valores no eixo X sem divisão
    plt.xticks(rotation=45)

    # Ajustar layout e mostrar o gráfico
    plt.tight_layout()
    plt.show()

except Exception as e:
    print("Erro ao executar a query:", e)


In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

# Configurar o nome do projeto explicitamente
PROJECT_ID = 'tcmodulo03'

# Criar o cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Query para obter dados de casos por região
query = """
SELECT
  regiao,
  percentual
FROM `tcmodulo03.PNAD_COVID.casos_por_regiao`
"""

# Executar a query e converter para DataFrame
df_regioes = client.query(query).to_dataframe()

# Criar o gráfico de pizza
plt.figure(figsize=(8, 8))
plt.pie(df_regioes['percentual'], labels=df_regioes['regiao'], autopct='%1.1f%%', startangle=140, colors=sns.color_palette("Set2", len(df_regioes)))
plt.title('Distribuição de Casos por Região')

# Exibir o gráfico
plt.axis('equal')  # Para garantir que o gráfico seja circular
plt.show()


In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar o nome do projeto explicitamente
PROJECT_ID = 'tcmodulo03'

# Criar o cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Query para obter dados de faixas etárias e percentual de casos
query = """
SELECT
  faixa_etaria,
  percentual
FROM `tcmodulo03.PNAD_COVID.view_faixas_etarias_positivos`
"""

# Executar a query e converter para DataFrame
df_faixas_etarias = client.query(query).to_dataframe()

# Exibir os dados obtidos
print(df_faixas_etarias)

# Criar o gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(x='faixa_etaria', y='percentual', data=df_faixas_etarias, palette='viridis')

# Configurar o título e rótulos do gráfico
plt.title('Percentual de Casos Positivos por Faixa Etária')
plt.xlabel('Faixa Etária')
plt.ylabel('Percentual (%)')

# Exibir o gráfico
plt.xticks(rotation=45)  # Para ajustar o texto das faixas etárias no eixo X
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# Configurar o nome do projeto explicitamente
PROJECT_ID = 'tcmodulo03'

# Criar o cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Query para obter os dados de doenças
query = """
SELECT
  tipo_doenca,
  total_casos,
  percentual
FROM `tcmodulo03.PNAD_COVID.homens_com_doencas`
"""

# Executar a query e converter para DataFrame
df_doencas = client.query(query).to_dataframe()

# Criar gráfico de rosca
plt.figure(figsize=(8, 8))
plt.pie(df_doencas['total_casos'], labels=df_doencas['tipo_doenca'], autopct='%1.1f%%', startangle=140,
        wedgeprops={'width': 0.6}, colors=sns.color_palette("Set2", len(df_doencas)))

# Ajustar o título e a posição
plt.title('Distribuição de Casos de Doenças entre Homens', fontsize=16, ha='center')

# Ajustar o layout para não sobrepor
plt.subplots_adjust(top=0.99)  # Aumenta o espaço entre o título e o gráfico

# Exibir o gráfico
plt.axis('equal')  # Para garantir que o gráfico seja circular
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# Configurar o nome do projeto explicitamente
PROJECT_ID = 'tcmodulo03'

# Criar o cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Query para obter os dados de doenças
query = """
SELECT
  tipo_doenca,
  total_casos,
  percentual
FROM `tcmodulo03.PNAD_COVID.mulheres_com_doencas`
"""

# Executar a query e converter para DataFrame
df_doencas = client.query(query).to_dataframe()

# Criar gráfico de rosca
plt.figure(figsize=(8, 8))
plt.pie(df_doencas['total_casos'], labels=df_doencas['tipo_doenca'], autopct='%1.1f%%', startangle=140,
        wedgeprops={'width': 0.6}, colors=sns.color_palette("Set2", len(df_doencas)))

# Ajustar o título e a posição
plt.title('Distribuição de Casos de Doenças entre Mulheres ', fontsize=16, ha='center')

# Ajustar o layout para não sobrepor
plt.subplots_adjust(top=0.99)  # Aumenta o espaço entre o título e o gráfico

# Exibir o gráfico
plt.axis('equal')  # Para garantir que o gráfico seja circular
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Dados simulados conforme o que você mencionou
dados = {
    "status_emprego": [
        "Não se aplica",
        "Não trabalha",
        "Trabalha (trabalhou ou fez bico na última semana)",
        "Trabalha (trabalhou ou fez bico na última semana)"
    ],
    "tipo_emprego": [
        "Não se aplica",
        "Não trabalha",
        "Trabalha em apenas um emprego",
        "Trabalha em mais de um emprego"
    ],
    "total_pessoas": [1157, 10425, 9906, 997],
    "percentual": [5.15, 46.36, 44.06, 4.43]
}

# Criar o DataFrame
df = pd.DataFrame(dados)

# Criar o gráfico de barras
fig, ax = plt.subplots(figsize=(10, 6))

# Definir as cores para cada barra
cores = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99']

# Plotar as barras
bars = ax.bar(df['status_emprego'] + " - " + df['tipo_emprego'], df['total_pessoas'], color=cores)

# Adicionar título e rótulos
ax.set_title('Distribuição de Emprego - Homens com Doenças', fontsize=16)
ax.set_ylabel('Número de Pessoas', fontsize=12)
ax.set_xlabel('Status e Tipo de Emprego', fontsize=12)

# Adicionar percentual nas barras
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, yval + 100, f'{yval}', ha='center', va='bottom', fontsize=10)

# Remover a legenda do eixo X
ax.set_xticks([])  # Isso remove as legendas (rótulos) do eixo X

# Criar a legenda à direita com quadrinhos coloridos
handles = [plt.Line2D([0], [0], color=c, lw=4) for c in cores]
labels = ['Não se aplica' , 'Não trabalha',
          'Trabalha apenas um emprego',
          'Trabalha em mais de um emprego']
plt.legend(handles, labels, title="Legenda", loc='center left', bbox_to_anchor=(1, 0.5))

# Ajustar o layout para não cortar nada
plt.tight_layout()

# Exibir o gráfico
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# Configurar o nome do projeto explicitamente
PROJECT_ID = 'tcmodulo03'

# Criar o cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Query para obter os dados
query = """
SELECT
    DB.ANO,
    DB.MES,
    SUM(CASE WHEN CAST(DB.B011 AS INT64) = 1 THEN 1 ELSE 0 END) AS Nao_fez_restricao,
    SUM(CASE WHEN CAST(DB.B011 AS INT64) = 2 THEN 1 ELSE 0 END) AS Reduziu_contato,
    SUM(CASE WHEN CAST(DB.B011 AS INT64) = 3 THEN 1 ELSE 0 END) AS Ficou_em_casa_necessidade,
    SUM(CASE WHEN CAST(DB.B011 AS INT64) = 4 THEN 1 ELSE 0 END) AS Ficou_rigorosamente_isolado,
    COUNT(*) AS Total_Respostas
FROM
    `tcmodulo03.PNAD_COVID.view_resultados_negativos` DB
WHERE
    CAST(DB.B011 AS INT64) IN (1, 2, 3, 4)
GROUP BY
    DB.ANO,
    DB.MES
ORDER BY
    DB.ANO,
    DB.MES;
"""

# Executando a consulta e armazenando o resultado em um DataFrame
df = client.query(query).to_dataframe()

# Mapeando os meses para garantir a ordem correta
meses_map = {'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4, 'maio': 5,
             'junho': 6, 'julho': 7, 'agosto': 8, 'setembro': 9, 'outubro': 10,
             'novembro': 11, 'dezembro': 12}

df['MES'] = df['MES'].map(meses_map)
df.sort_values(by=['ANO', 'MES'], inplace=True)

# Renomeando os meses para legibilidade
meses_legiveis = {1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio',
                  6: 'Junho', 7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro',
                  11: 'Novembro', 12: 'Dezembro'}

df['MES'] = df['MES'].map(meses_legiveis)

# Plotando o gráfico de barras empilhadas
plt.figure(figsize=(12, 8))
sns.set(style="whitegrid")

# Ajustando as cores
cores = ['#4E79A7', '#F28E2B', '#E15759', '#76B7B2']

# Criando as barras empilhadas com largura ajustada
bar1 = plt.bar(df['MES'], df['Nao_fez_restricao'], width=0.4, label='Não fez restrição', color=cores[0])
bar2 = plt.bar(df['MES'], df['Reduziu_contato'], width=0.4, bottom=df['Nao_fez_restricao'], label='Reduziu contato', color=cores[1])
bar3 = plt.bar(df['MES'], df['Ficou_em_casa_necessidade'], width=0.4,
               bottom=df['Nao_fez_restricao'] + df['Reduziu_contato'],
               label='Ficou em casa por necessidade', color=cores[2])
bar4 = plt.bar(df['MES'], df['Ficou_rigorosamente_isolado'], width=0.4,
               bottom=df['Nao_fez_restricao'] + df['Reduziu_contato'] + df['Ficou_em_casa_necessidade'],
               label='Ficou rigorosamente isolado', color=cores[3])

# Adicionando títulos e rótulos
plt.title('Medidas de segurança', fontsize=14, fontweight='bold')
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Número de Respostas', fontsize=12)
plt.legend(loc='upper left')
plt.xticks(rotation=45)  # Rotaciona os nomes dos meses para melhor visualização

# Ajusta layout para não cortar elementos
plt.tight_layout()

# Exibir o gráfico
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# Configurar o projeto no BigQuery
PROJECT_ID = 'tcmodulo03'
client = bigquery.Client(project=PROJECT_ID)

# Query SQL
query = """
SELECT
    view_resultados_positivos.C011A11 AS FaixaSalarial,
    MAX(view_resultados_positivos.C011A12) AS MaxSalario,
    MIN(view_resultados_positivos.C011A12) AS MinSalario,
    AVG(view_resultados_positivos.C011A12) AS MediaSalarial,
    COUNT(*) AS QuantidadePessoas
FROM
    `tcmodulo03.PNAD_COVID.view_resultados_positivos` AS view_resultados_positivos
JOIN
    `tcmodulo03.PNAD_COVID.faixaSalarial` AS faixaSalarial
ON
    view_resultados_positivos.C011A11 = faixaSalarial.FaixaSalarial
WHERE
    view_resultados_positivos.C011A12 IS NOT NULL
GROUP BY
    view_resultados_positivos.C011A11
ORDER BY
    QuantidadePessoas DESC;
"""

# Executar a consulta e carregar os resultados em um DataFrame
df = client.query(query).to_dataframe()

# Ordenar os dados por Faixa Salarial (garantir que fique de 1 a 4)
df['FaixaSalarial'] = pd.Categorical(df['FaixaSalarial'], categories=sorted(df['FaixaSalarial'].unique()), ordered=True)
df = df.sort_values(by='FaixaSalarial')

# Gráfico de barras - Volume de Pessoas por Faixa Salarial
plt.figure(figsize=(12, 8))
sns.set(style="whitegrid")

ax = sns.barplot(
    x="FaixaSalarial",
    y="QuantidadePessoas",
    data=df,
    palette="Blues_d"
)

# Títulos e legendas
ax.set_title("Volume de Pessoas por Faixa Salarial", fontsize=14)
ax.set_xlabel("Faixa Salarial", fontsize=12)
ax.set_ylabel("Quantidade de Pessoas", fontsize=12)

# Adicionar legenda com o range da faixa salarial
handles = []
# Ordenando a faixa salarial de 1 a 4
for _, row in df.sort_values('FaixaSalarial').iterrows():
    label = f"{row['FaixaSalarial']}: R${row['MinSalario']} - R${row['MaxSalario']}"
    handles.append(plt.Line2D([0], [0], color="blue", lw=0, label=label))

ax.legend(handles=handles, loc="upper right", fontsize=10)

# Rotacionar os rótulos do eixo X
plt.xticks(rotation=45)

# Mostrar o gráfico
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# Configurar o projeto no BigQuery
PROJECT_ID = 'tcmodulo03'
client = bigquery.Client(project=PROJECT_ID)


# Query para buscar os dados, excluindo "Outros/Não Aplicável"
query = """
SELECT
    local_ajuda,
    total_geral
FROM
    `tcmodulo03.PNAD_COVID.view_busca_ajuda`
WHERE
    local_ajuda != 'Outros/Não Aplicável';
"""

# Executando a consulta e armazenando os dados em um DataFrame
df = client.query(query).to_dataframe()

# Exibindo as primeiras linhas dos dados para conferirmos
print(df.head())

# Criando gráfico de rosca (donut chart)
quantidades = df['total_geral']
rótulos = df['local_ajuda']

# Criando o gráfico de rosca
plt.figure(figsize=(8, 8))
plt.pie(quantidades, labels=rótulos, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.4})

# Adicionando o título ao gráfico
plt.title('Distribuição de Local de Ajuda e Quantidade por Local')

# Exibindo o gráfico
plt.show()
